# TRAINER 문서의 Trainer flags
### 본 문서는 PyTorch Lightning의 [공식 가이드](https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags)의 한글 번역본입니다. (옮긴이 [dnap512](https://github.com/dnap512), 21.7.15)

#### 이번 문서는 너무 길어서 따로 분리했습니다. 원문에는 아래의 거의 모든 항목마다 동영상이 있으니 참고하시면 좋습니다.

## accelerator

사용할 accelerator backend(이전에는 distribution_backend로 알려짐).

- (`'dp'`)는 DataParallel(동일한 시스템의 GPU 간 분할 배치)입니다.
- (`'ddp'`)는 DistributedDataParallel입니다(각 노드의 각 GPU는 grads를 훈련하고 동기화함).
- (`'ddp_cpu'`)는 CPU에서 DistributedDataParallel입니다('ddp'와 동일하지만 GPU를 사용하지 않습니다. 다중 노드 CPU 교육 또는 단일 노드 디버깅에 유용합니다. 단일 노드에서 속도 향상을 제공하지 **않습니다**. Torch는 이미 단일 시스템에서 여러 CPU를 효율적으로 사용하고 있습니다.)
- (`'ddp2'`) 노드의 dp, 노드 전체의 ddp. negative sample의 수를 늘리는 것과 같은 작업에 유용합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(accelerator=None)

# dp = DataParallel
trainer = Trainer(gpus=2, accelerator='dp')

# ddp = DistributedDataParallel
trainer = Trainer(gpus=2, num_nodes=2, accelerator='ddp')

# ddp2 = DistributedDataParallel + dp
trainer = Trainer(gpus=2, num_nodes=2, accelerator='ddp2')

> <span style='color:blue'>NOTE:</span> 이 옵션은 TPU에 적용되지 않습니다. TPU는 기본적으로 'ddp'를 사용합니다(각 코어에 대해).

필요에 맞게 기존 가속기를 subclassing하여 하드웨어 동작을 수정할 수도 있습니다.

다음 예를 확인하세요

In [ ]:
class MyOwnAcc(Accelerator):
    ...

Trainer(accelerator=MyOwnAcc())

> <span style='color:red'>경고:</span> 사용자 지정 가속기를 전달하는 것은 실험적이며, 완전한 호환성을 위한 작업이 진행 중입니다.

## accumulate_grad_batches

k 배치마다 또는 dict에 설정된 대로 grads를 누적합니다. 또한 Trainer는 설정한 배치의 마지막 번째 배치에서 `optimizer.step()`을 호출합니다. (역자 주: 메모리의 한계로 배치 사이즈를 줄여야 할 때, 이 옵션이 학습 상으로는 배치를 키울 수 있게 해줍니다.)

In [ ]:
# default used by the Trainer (no accumulation)
trainer = Trainer(accumulate_grad_batches=1)

# accumulate every 4 batches (effective batch size is batch*4)
trainer = Trainer(accumulate_grad_batches=4)

# no accumulation for epochs 1-4. accumulate 3 for epochs 5-10. accumulate 20 after that
trainer = Trainer(accumulate_grad_batches={5: 3, 10: 20})

## amp_backend

Use PyTorch AMP (‘native’) (available PyTorch 1.6+), or NVIDIA apex (‘apex’).

In [ ]:
# using PyTorch built-in AMP, default used by the Trainer
trainer = Trainer(amp_backend='native')

# using NVIDIA Apex
trainer = Trainer(amp_backend='apex')

## amp_level

The optimization level to use (O1, O2, etc…) for 16-bit GPU precision (using NVIDIA apex under the hood).

Check [NVIDIA apex docs](https://nvidia.github.io/apex/amp.html#opt-levels) for level

In [ ]:
# default used by the Trainer
trainer = Trainer(amp_level='O2')

## auto_scale_batch_size

훈련 전에 메모리에 맞는 가장 큰 배치 크기를 자동으로 찾으려고 시도합니다.



In [ ]:
# default used by the Trainer (no scaling of batch size)
trainer = Trainer(auto_scale_batch_size=None)

# run batch size scaling, result overrides hparams.batch_size
trainer = Trainer(auto_scale_batch_size='binsearch')

# call tune to find the batch size
trainer.tune(model)

## auto_select_gpus

활성화되고 gpus가 정수인 경우 사용 가능한 GPU를 자동으로 선택합니다. 이는 GPU가 "독점 모드"로 구성되어 한 번에 하나의 프로세스만 GPU에 액세스할 수 있도록 구성된 경우에 특히 유용합니다.

In [ ]:
# no auto selection (picks first 2 gpus on system, may fail if other process is occupying)
trainer = Trainer(gpus=2, auto_select_gpus=False)

# enable auto selection (will find two available gpus on system)
trainer = Trainer(gpus=2, auto_select_gpus=True)

# specifies all GPUs regardless of its availability
Trainer(gpus=-1, auto_select_gpus=False)

# specifies all available GPUs (if only one GPU is not occupied, uses one gpu)
Trainer(gpus=-1, auto_select_gpus=True)

## auto_lr_find

`training.tune()`을 호출할 때, 최적의 초기 학습률을 찾기 위해 [Learning rate finder algorithm](https://arxiv.org/abs/1506.01186)을 실행합니다.

In [ ]:
# default used by the Trainer (no learning rate finder)
trainer = Trainer(auto_lr_find=False)

################

# run learning rate finder, results override hparams.learning_rate
trainer = Trainer(auto_lr_find=True)

# call tune to find the lr
trainer.tune(model)

################

# run learning rate finder, results override hparams.my_lr_arg
trainer = Trainer(auto_lr_find='my_lr_arg')

# call tune to find the lr
trainer.tune(model)

> <span style='color:blue'>NOTE:</span> [Learning rate finder algorithm 가이드](https://pytorch-lightning.readthedocs.io/en/latest/advanced/lr_finder.html)를 살펴보세요. 


## benchmark

true이면 `cudnn.benchmark`를 활성화합니다. 이 플래그는 입력 크기가 변경되지 않으면 시스템 속도를 높일 수 있습니다. 하지만 입력 크기가 변경되면 시스템이 느려질 수 있습니다.

속도 향상은 cudnn auto-tunner가 하드웨어에 가장 적합한 알고리즘을 찾을 수 있도록 하는 데서 비롯됩니다[discussion 참조](https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936).

In [ ]:
# default used by the Trainer
trainer = Trainer(benchmark=False)

## deterministic

true이면 cudnn.deterministic을 활성화합니다. 시스템을 느리게 만들 수 있지만 재현성을 보장합니다. 또한 `$HOROVOD_FUSION_THRESHOLD=0`을 설정합니다.

자세한 내용은 [pytorch docs](https://pytorch.org/docs/stable/notes/randomness.html)를 확인하세요.

In [ ]:
# default used by the Trainer
trainer = Trainer(deterministic=False)

## callbacks

[`Callback`](https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.callbacks.Callback.html#pytorch_lightning.callbacks.Callback) 목록을 추가합니다. Callback은 여기에 정의된 순서대로 순차적으로 실행됩니다(모든 상태가 체크포인트에 저장되도록 다른 모든 콜백 이후에 실행되는 [`ModelCheckpoint`] Callback 제외).

In [ ]:
# a list of callbacks
callbacks = [PrintCallback()]
trainer = Trainer(callbacks=callbacks)

In [ ]:
from pytorch_lightning.callbacks import Callback

class PrintCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        print("Training is started!")
    def on_train_end(self, trainer, pl_module):
        print("Training is done.")

모델별 콜백은 [`configure_callbacks()`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.lightning.html#pytorch_lightning.core.lightning.LightningModule.configure_callbacks)를 통해 `LightningModule` 내부에 추가할 수도 있습니다. 이 Hook에서 반환된 콜백은 처음에 Trainer argument에 제공된 목록을 확장하고 동일한 유형의 두 개 이상이 있는 경우 트레이너 콜백을 대체합니다. [`ModelCheckpoint`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.callbacks.model_checkpoint.html#pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint) 콜백은 항상 마지막에 실행됩니다.

## check_val_every_n_epoch

매 에포크마다 Validation 수행합니다.


In [ ]:
# default used by the Trainer
trainer = Trainer(check_val_every_n_epoch=1)

# run val loop every 10 training epochs
trainer = Trainer(check_val_every_n_epoch=10)

## checkpoint_callback

기본적으로 Lightning은 현재 작업 디렉토리에 체크포인트(마지막 Training epoch의 state와 함께)를 저장하고 체크포인트는 모델에서 사용하는 모든 매개변수의 정확한 값을 캡처합니다. 자동 체크포인트를 비활성화하려면 False로 설정하십시오.

In [ ]:
# default used by Trainer
trainer = Trainer(checkpoint_callback=True)

# turn off automatic checkpointing
trainer = Trainer(checkpoint_callback=False)

[`ModelCheckpoint`](https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.callbacks.ModelCheckpoint.html#pytorch_lightning.callbacks.ModelCheckpoint) 콜백을 초기화하고 [`callbacks`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.trainer.trainer.html#pytorch_lightning.trainer.trainer.Trainer.params.callbacks) 목록에 추가하여 기본 동작을 재정의할 수 있습니다. 체크포인트를 사용자 정의하는 방법은 [`Saving and Loading Weights docs`](https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html)를 참조하십시오.

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor='val_loss')

# Add your callback to the callbacks list
trainer = Trainer(callbacks=[checkpoint_callback])

> <span style='color:red'>경고:</span> ModelCheckpoint 인스턴스를 이 인수에 전달하는 것은 v1.1부터 deprecated 되었으며 v1.3부터는 지원되지 않습니다. 대신 콜백 인수를 사용하세요.

## default_root_dir

로거 또는 [`pytorch_lightning.callbacks.ModelCheckpoint`] 콜백이 전달되지 않은 경우 로그 및 가중치가 저장되는 기본 경로입니다. 특정 클러스터에서는 로그와 체크포인트가 저장되는 위치를 분리해야 할 수 있습니다. 그렇지 않은 경우 편의를 위해 이 인수를 사용합니다. 경로는 로컬 경로 또는 s3://bucket/path 또는 'hdfs://path/'와 같은 원격 경로일 수 있습니다. 원격 파일 경로를 사용하려면 자격 증명을 설정해야 합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(default_root_dir=os.getcwd())

## fast_dev_run

모든 버그(즉, 일종의 단위 테스트)를 찾기 위한 목적으로, n으로 설정된 경우 n번의 배치 테스트를 수행하고, True로 설정된 경우 1번 배치 테스트를 수행합니다.

내부에서 단일 배치로 fast_dev_run을 실행할 때 Psuedocode는 다음과 같습니다.

In [ ]:
# loading
__init__()
prepare_data

# test training step
training_batch = next(train_dataloader)
training_step(training_batch)

# test val step
val_batch = next(val_dataloader)
out = validation_step(val_batch)
validation_epoch_end([out])

In [ ]:
# default used by the Trainer
trainer = Trainer(fast_dev_run=False)

# runs 1 train, val, test batch and program ends
trainer = Trainer(fast_dev_run=True)

# runs 7 train, val, test batches and program ends
trainer = Trainer(fast_dev_run=7)

> <span style='color:blue'>NOTE:</span> 이 인수는 `limit_train/val/test_batches`와 약간 다릅니다. 이 인수를 설정하면 tunner, checkpoint callbacks, early stopping callbacks, logger 및 `LearningRateLogger`와 같은 logger callback이 비활성화되고 1 epoch 동안만 실행됩니다. 이것은 디버깅 목적으로만 사용해야 합니다. `limit_train/val/test_batches`는 배치 수만 제한하고 아무 것도 비활성화하지 않습니다.

## flush_logs_every_n_steps

로깅 주기를 정합니다. [logging docs](https://pytorch-lightning.readthedocs.io/en/latest/extensions/logging.html)를 참고하세요.

In [ ]:
# default used by the Trainer
trainer = Trainer(flush_logs_every_n_steps=100)

## gpus

- 학습에 사용할 gpu 수 (int)
- 아니면 학습에 사용할 gpu의 번호 (list)
- string값으로도 적용할 수 있습니다.

In [ ]:
# default used by the Trainer (ie: train on CPU)
trainer = Trainer(gpus=None)

# equivalent
trainer = Trainer(gpus=0)

In [ ]:
# int: train on 2 gpus
trainer = Trainer(gpus=2)

# list: train on GPUs 1, 4 (by bus ordering)
trainer = Trainer(gpus=[1, 4])
trainer = Trainer(gpus='1, 4') # equivalent

# -1: train on all gpus
trainer = Trainer(gpus=-1)
trainer = Trainer(gpus='-1') # equivalent

# combine with num_nodes to train on multiple GPUs across nodes
# uses 8 gpus in total
trainer = Trainer(gpus=2, num_nodes=4)

# train only on GPUs 1 and 4 across nodes
trainer = Trainer(gpus=[1, 4], num_nodes=4)

## gradient_clip_val

Gradient clipping 값을 정합니다. 0으로 적용하면 clipping하지 않는 것을 뜻합니다.


In [ ]:
# default used by the Trainer
trainer = Trainer(gradient_clip_val=0.0)

## limit_train_batches

수행할 Training set의 양을 제한하여 적용합니다. Epoch가 끝날 때 발생하는 무언가를 디버깅하거나 테스트할 때 유용합니다.



In [ ]:
# default used by the Trainer
trainer = Trainer(limit_train_batches=1.0)

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_train_batches=1.0)

# run through only 25% of the training set each epoch
trainer = Trainer(limit_train_batches=0.25)

# run through only 10 batches of the training set each epoch
trainer = Trainer(limit_train_batches=10)

## limit_test_batches

수행할 Test set의 양을 제한하여 적용합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_test_batches=1.0)

# run through only 25% of the test set each epoch
trainer = Trainer(limit_test_batches=0.25)

# run for only 10 batches
trainer = Trainer(limit_test_batches=10)

여러개의 Test dataloader의 경우 제한이 각 데이터 로더에 개별적으로 적용됩니다.

## limit_val_batches

수행할 Valid set의 양을 제한하여 적용합니다. Epoch가 끝날 때 발생하는 무언가를 디버깅하거나 테스트할 때 유용합니다.

여러개의 Valid dataloader의 경우 제한이 각 데이터 로더에 개별적으로 적용됩니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(limit_val_batches=1.0)

# run through only 25% of the validation set each epoch
trainer = Trainer(limit_val_batches=0.25)

# run for only 10 batches
trainer = Trainer(limit_val_batches=10)

## log_every_n_steps

로깅 행을 실제 디스크에 얼마나 자주 추가할지 정합니다.(매 배치마다 실제 디스크에 로깅하게 되면 그만틈 느리겠죠)

In [ ]:
# default used by the Trainer
trainer = Trainer(log_every_n_steps=50)

## log_gpu_memory

옵션 목록

- None
- 'min_max'
- 'all'


In [ ]:
# default used by the Trainer
trainer = Trainer(log_gpu_memory=None)

# log all the GPUs (on master node only)
trainer = Trainer(log_gpu_memory='all')

# log only the min and max memory on the master node
trainer = Trainer(log_gpu_memory='min_max')

> <span style='color:blue'>NOTE:</span> `nvidia-smi`의 출력을 사용하기 때문에 성능이 느려질 수 있습니다.

## logger

실험 추적을 위한 [`Logger`](https://pytorch-lightning.readthedocs.io/en/latest/common/loggers.html)(or iterable collection of loggers). True 값은 아래의 기본 `TensorBoardLogger`를 사용합니다. False는 로깅을 비활성화합니다.

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

# default logger used by trainer
logger = TensorBoardLogger(
    save_dir=os.getcwd(),
    version=1,
    name='lightning_logs'
)
Trainer(logger=logger)

## max_epochs



In [ ]:
# default used by the Trainer
trainer = Trainer(max_epochs=1000)

## min_epochs

최소한 훈련할 epochs 수를 정합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(min_epochs=1)

## max_steps

최대 Iteration 수를 정합니다.

In [ ]:
# Default (disabled)
trainer = Trainer(min_steps=None)

# Run at least for 100 steps (disable min_epochs)
trainer = Trainer(min_steps=100, min_epochs=0)

## max_time

훈련을 위한 최대 시간을 설정합니다. 지정 시간이 지나면 훈련은 중단됩니다. 사용자 지정 가능한 옵션의 경우 타이머 콜백을 사용하세요!

In [ ]:
# Default (disabled)
trainer = Trainer(max_time=None)

# Stop after 12 hours of training or when reaching 10 epochs (string)
trainer = Trainer(max_time="00:12:00:00", max_epochs=10)

# Stop after 1 day and 5 hours (dict)
trainer = Trainer(max_time={"days": 1, "hours": 5})

`max_time`이 `min_steps` 또는 `min_epochs`와 함께 사용되는 경우 `min_*` 조건이 항상 더 높은 우선순위에 있습니다.

## num_nodes

분산 학습을 목적으로 사용할 GPU 노드 수입니다

In [ ]:
# default used by the Trainer
trainer = Trainer(num_nodes=1)

# to train on 8 nodes
trainer = Trainer(num_nodes=8)

## num_processes

학습할 프로세스의 수입니다. `accelerator="ddp"`를 사용할 때 GPU 수로 자동 설정됩니다. `Accelerator="ddp_cpu"`를 사용하여 GPU가 없는 머신에서 분산 교육을 모방할 때 1보다 큰 숫자로 설정합니다. 이것은 디버깅에 유용하지만 단일 프로세스 Torch가 이미 여러 CPU를 효율적으로 사용하고 있기 때문에 속도 향상을 제공하지 않습니다.

In [ ]:
# Simulate DDP for debugging on your GPU-less laptop
trainer = Trainer(accelerator="ddp_cpu", num_processes=2)

## num_sanity_val_steps

Sanity check는 훈련을 시작하기 전에 n개의 val 배치를 수행합니다. 이렇게 하면 첫 번째  validation을 기다릴 필요 없이 유효성 검사의 모든 버그를 잡을 수 있습니다. Trainer는 기본적으로 2개 배치를 사용합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(num_sanity_val_steps=2)

# turn it off
trainer = Trainer(num_sanity_val_steps=0)

# check all validation data
trainer = Trainer(num_sanity_val_steps=-1)

이 옵션은 num_sanity_val_steps=0이 아닌 경우 val 데이터로더를 리셋합니다.

## overfit_batches

Training set에서 지정한 만큼의 데이터를 사용합니다. 0이 아니면 검증 및 테스트에 동일한 훈련 세트를 사용합니다. 훈련 데이터 로더가 shuffle=True이면 Lightning이 자동으로 비활성화합니다.

빠르게 디버깅하거나 의도적으로 overfitting을 시도하는 데 유용합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(overfit_batches=0.0)

# use only 1% of the train set (and use the train set for val and test)
trainer = Trainer(overfit_batches=0.01)

# overfit on 10 of the same batches
trainer = Trainer(overfit_batches=10)

## plugins

[plugins](https://pytorch-lightning.readthedocs.io/en/latest/extensions/plugins.html#plugins)는 여러분이 임의의  backends, precision libraries, clusters 등에 연결하게 해줍니다.

- [DDP](https://pytorch-lightning.readthedocs.io/en/latest/advanced/multi_gpu.html#multi-gpu)
- [TorchElastic](https://pytorch.org/elastic/0.2.2/index.html)
- [Apex](https://pytorch-lightning.readthedocs.io/en/latest/guides/speed.html#amp)

여러분만의 behavior를 정의하려면 관련 클래스를 하위 클래스로 만들고 전달하세요. 다음은 여러분만의 [`ClusterEnvironment`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.plugins.environments.ClusterEnvironment.html#pytorch_lightning.plugins.environments.ClusterEnvironment)를 연결하는 예입니다.


In [ ]:
from pytorch_lightning.plugins.environments import ClusterEnvironment

class MyCluster(ClusterEnvironment):

    def master_address(self):
        return your_master_address

    def master_port(self):
        return your_master_port

    def world_size(self):
        return the_world_size

trainer = Trainer(plugins=[MyCluster()], ...)

## prepare_data_per_node

True이면 모든 노드에 대해 `LOCAL_RANK=0`에서 `prepare_data()`를 호출합니다. False라면 `NODE_RANK=0`, `LOCAL_RANK=0`에서만 호출됩니다.

In [ ]:
# default
Trainer(prepare_data_per_node=True)

# use only NODE_RANK=0, LOCAL_RANK=0
Trainer(prepare_data_per_node=False)

## precision

Lightning은 double precision(64), full precision(32) 또는 half precision(16) 훈련을 지원합니다.

Half precision 또는 mixed precision은 32비트 및 16비트 부동 소수점을 결합하여 모델 훈련 중에 메모리 공간을 줄이는 것입니다. 그 결과 성능이 향상되어 최신 GPU에서 +3배의 속도 향상을 달성할 수 있습니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(precision=32, gpus=1)

# 16-bit precision
trainer = Trainer(precision=16, gpus=1)

# 64-bit precision
trainer = Trainer(precision=64, gpus=1)

> <span style='color:blue'>NOTE:</span> TPU 상에서 돌아갈 때, torch.float16이 사용되지만 텐서 인쇄에는 여전히 torch.float32가 표시됩니다.

> <span style='color:blue'>NOTE:</span> 16bit precision은 CPU 상에서 지원되지 않습니다.

> <span style='color:red'>PYTORCH 1.6+를 사용할 때 LIGHTNING은 네이티브 AMP 구현을 사용하여 16bit precision을 지원합니다. 1.6 버전 이하의 PYTORCH 16bit precision은 NVIDIA APEX 라이브러리에서 지원됩니다.</span>

## process_position

Progress bar를 주문합니다. 동일한 노드에서 여러 트레이너를 실행할 때 유용합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(process_position=0)

> <span style='color:blue'>NOTE:</span> 사용자 정의 콜백이 `callbacks`에 전달되면 이 인수는 무시됩니다.

## profiler

모델 훈련 중 각각의 단계를 프로파일링하고 병목 현상을 찾아내는 데 도움이 됩니다.

자세한 사항은 [profiler docs](https://pytorch-lightning.readthedocs.io/en/latest/advanced/profiler.html)를 참고하세요!

In [ ]:
from pytorch_lightning.profiler import SimpleProfiler, AdvancedProfiler

# default used by the Trainer
trainer = Trainer(profiler=None)

# to profile standard training events, equivalent to `profiler=SimpleProfiler()`
trainer = Trainer(profiler="simple")

# advanced profiler for function-level stats, equivalent to `profiler=AdvancedProfiler()`
trainer = Trainer(profiler="advanced")

## progress_bar_refresh_rate

Progress bar를 얼마나 자주 새로고침 할지 정합니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(progress_bar_refresh_rate=1)

# disable progress bar
trainer = Trainer(progress_bar_refresh_rate=0)

Note:
- Google Colab 노트북에서는 화면 새로 고침 빈도로 인해 더 빠른 새로 고침 빈도(낮은 숫자)가 충돌하는 것으로 알려져 있습니다. 사용자가 값을 제공하지 않으면 Lightning은 이러한 환경에서 이 값을 20으로 설정합니다.

- 사용자 정의 콜백이 `callbacks`에 전달되면 이 인수는 무시됩니다.

## reload_dataloaders_every_n_epochs

양의 정수(n)로 설정하면 n epoch마다 데이터로더를 다시 로드합니다.


In [ ]:
# if 0 (default)
train_loader = model.train_dataloader()
for epoch in epochs:
    for batch in train_loader:
        ...

# if a positive integer
for epoch in epochs:
    if not epoch % reload_dataloaders_every_n_epochs:
        train_loader = model.train_dataloader()
    for batch in train_loader:

## replace_sampler_ddp

[`DistributedSampler`](https://pytorch.org/docs/stable/data.html#torch.utils.data.distributed.DistributedSampler)의 자동 추가를 활성화합니다. PyTorch에서는 TPU 또는 다중 노드와 같은 분산 설정에서 사용해야 합니다. 샘플러는 각 GPU가 데이터의 적절한 부분을 볼 수 있도록 합니다. 기본적으로 train 샘플러에는 `shuffle=True`가 추가되고 val/test 샘플러에는 shuffle=False가 추가됩니다. 이를 사용자 정의하려면 `replace_sampler_ddp=False`를 설정하고 여러분만의 분산 샘플러를 추가할 수 있습니다. `replace_sampler_ddp=True`이고 분산 샘플러가 이미 추가된 경우 Lightning이 기존 샘플러를 대체하지 않습니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(replace_sampler_ddp=True)

False로 설정하면 여러분만의 분산 샘플러를 추가해야 합니다

In [ ]:
# in your LightningModule or LightningDataModule
def train_dataloader(self):
    # default used by the Trainer
    sampler = torch.utils.data.distributed.DistributedSampler(dataset, shuffle=True)
    dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
    return dataloader

> <span style='color:blue'>NOTE:</span> Iterable dataset에 대해서 자동으로 적용하지 않습니다.

## resume_from_checkpoint

특정 체크포인트에서 훈련을 재개하려면 여기에 경로를 입력하세요. Epoch 중간의 체크포인트에서 다시 시작하면 다음 에포크 시작 부분부터 훈련이 시작됩니다.

In [ ]:
# default used by the Trainer
trainer = Trainer(resume_from_checkpoint=None)

# resume from a specific checkpoint
trainer = Trainer(resume_from_checkpoint='some/path/to/my_checkpoint.ckpt')

## sync_batchnorm

모든 GPU에서 batchnorm 층간의 동기화를 활성화합니다.

In [ ]:
trainer = Trainer(sync_batchnorm=True)

## track_grad_norm

- 추적하지 않음 (-1)
- 특정 norm에 대해 추적함 (2면 2-norm)


In [ ]:
# default used by the Trainer
trainer = Trainer(track_grad_norm=-1)

# track the 2-norm
trainer = Trainer(track_grad_norm=2)

## tpu_cores

- 학습할 TPU 코어 수(1 또는 8).
- 학습할 TPU 코어 \[1-8\]

단일 TPU v2 또는 v3에는 8개의 코어가 있습니다. TPU 포드에는 최대 2048개의 코어가 있습니다. POD 슬라이스는 요청한 만큼의 코어를 얻을 수 있음을 의미합니다.

효과적인 배치 크기는 batch_size * 총 tpu 코어입니다.

이 매개변수는 1 또는 8일 수 있습니다.

In [ ]:
# your_trainer_file.py

# default used by the Trainer (ie: train on CPU)
trainer = Trainer(tpu_cores=None)

# int: train on a single core
trainer = Trainer(tpu_cores=1)

# list: train on a single selected core
trainer = Trainer(tpu_cores=[2])

# int: train on all cores few cores
trainer = Trainer(tpu_cores=8)

# for 8+ cores must submit via xla script with
# a max of 8 cores specified. The XLA script
# will duplicate script onto each TPU in the POD
trainer = Trainer(tpu_cores=8)

8개 이상의 코어(즉, POD)에서 훈련하려면 xla_dist 스크립트를 사용하여 이 스크립트를 제출하십시오.

```bash
python -m torch_xla.distributed.xla_dist
--tpu=$TPU_POD_NAME
--conda-env=torch-xla-nightly
--env=XLA_USE_BF16=1
-- python your_trainer_file.py
```


## truncated_bptt_steps

truncated_bptt_steps는 훨씬 긴 시퀀스의 k 단계마다 역전파를 수행합니다.

이것이 활성화되면 배치가 자동으로 잘리고 trainer가 잘린 Backprop을 적용합니다.

[(Williams et al. “An efficient gradient-based algorithm for on-line training of recurrent network trajectories.”)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.7941&rep=rep1&type=pdf)

In [ ]:
# default used by the Trainer (ie: disabled)
trainer = Trainer(truncated_bptt_steps=None)

# backprop every 5 steps in a batch
trainer = Trainer(truncated_bptt_steps=5)

> <span style='color:blue'>NOTE:</span> 배치에 시퀀스 차원이 있는지 확인하세요.

Lightning은 time dimension에 따라 배치를 분할합니다.


In [ ]:
# we use the second as the time dimension
# (batch, time, ...)
sub_batch = batch[0, 0:t, ...]

이 기능을 사용하려면 hidden state가 assign된 `hiddens` parameter를 포함하도록 LightningModule의 `pytorch_lightning.core.LightningModule.training_step()`을 업데이트해야 합니다.

In [ ]:
# Truncated back-propagation through time
def training_step(self, batch, batch_idx, hiddens):
    # hiddens are the hiddens from the previous truncated backprop step
    out, hiddens = self.lstm(data, hiddens)
    return {
        "loss": ...,
        "hiddens": hiddens
    }

## val_check_interval

한 개의 training epoch 내에서 valid를 수행하는 빈도입니다. float 또는 int로 지정할 수 있습니다.

- (float): Training epoch 내에서 자주 valid 하는 정도
- (int): n steps 마다 valid 수행



In [ ]:
# default used by the Trainer
trainer = Trainer(val_check_interval=1.0)

# check validation set 4 times during a training epoch
trainer = Trainer(val_check_interval=0.25)

# check validation set every 1000 training batches
# use this when using iterableDataset and your dataset has no length
# (ie: production cases with streaming data)
trainer = Trainer(val_check_interval=1000)

In [ ]:
# Here is the computation to estimate the total number of batches seen within an epoch.

# Find the total number of train batches
total_train_batches = total_train_samples // (train_batch_size * world_size)

# Compute how many times we will call validation during the training loop
val_check_batch = max(1, int(total_train_batches * val_check_interval))
val_checks_per_epoch = total_train_batches / val_check_batch

# Find the total number of validation batches
total_val_batches = total_val_samples // (val_batch_size * world_size)

# Total number of batches run
total_fit_batches = total_train_batches + total_val_batches

## weights_save_path

만약 지정하고 싶다면 가중치를 어디에 저장할지 지정하세요.


In [ ]:
# default used by the Trainer
trainer = Trainer(weights_save_path=os.getcwd())

# save to your custom path
trainer = Trainer(weights_save_path='my/path')

In [ ]:
# if checkpoint callback used, then overrides the weights path
# **NOTE: this saves weights to some/path NOT my/path
checkpoint = ModelCheckpoint(dirpath='some/path')
trainer = Trainer(
    callbacks=[checkpoint],
    weights_save_path='my/path'
)

## weights_summary

훈련이 시작될 때 가중치에 대한 요약을 출력합니다. (옵션: 'full', 'top', None)

In [ ]:
# default used by the Trainer (ie: print summary of top level modules)
trainer = Trainer(weights_summary='top')

# print full summary of all modules and submodules
trainer = Trainer(weights_summary='full')

# don't print a summary
trainer = Trainer(weights_summary=None)